# Divisiveness Scoring

I wanted to figure out a way to mathamatically offer secondary title to the participants with the most divisive tastes in the High Street Music league. This notebook uses the music league data downloads and calculates a divisiveness score based on the strength of voters' reactions to each submission and how much these votes cancel eachother out.

The current methodology is:
- Where $ v $ is an individual vote

For each round, each submission scores with the following:
$$\text{Sentiment Strength Score} = \sum_{i=1}^{n} |v_i|$$
$$\text{Distance} = |\sum_{i=1}^{n} v |$$
$$\text{Weight} = Percentile Rank(\text{Distance})$$
The final score for any submission within a round is:
$$\text{Score} = \text{Sentiment Strength Score} * \text{Weight} $$

Scores are totalled for each submitter across all rounds to determine who has the most divisive taste.


In [71]:
#import libraries
import pandas as pd

In [72]:
#Pull in data from Music League format
votes = pd.read_csv('data/votes.csv')
rounds = pd.read_csv('data/rounds.csv')
competitors = pd.read_csv('data/competitors.csv')
submissions = pd.read_csv('data/submissions.csv')

In [73]:
#Attatch song submitter to each vote
submitters = submissions[['Submitter ID', 'Spotify URI', 'Round ID']]
votes = pd.merge(votes, submitters, how='outer', on= ['Spotify URI', 'Round ID'])

In [74]:
#Get strength scores by taking the absolute value of each assigned point and summing them.
#Get the Distance from zero by taking the absolute value of the total points
votes['Strength Score'] = votes['Points Assigned'].map(abs)

df = votes.groupby(['Round ID', 'Submitter ID'])['Points Assigned', 'Strength Score'].sum()

df['Distance'] = df['Points Assigned'].map(abs)

df.reset_index(inplace=True)


/var/folders/kz/8hthyz657tlbhl0x92mmdj2r0000gn/T/ipykernel_4305/1748230846.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df = votes.groupby(['Round ID', 'Submitter ID'])['Points Assigned', 'Strength Score'].sum()


In [75]:
#Get a percentile rank score for the distance from zero for each submission within a round
distance_weight = df.groupby('Round ID')['Distance'].rank(ascending=False, pct=True).rename('Weight')

#Add these weights to the data frame
df = df.join(distance_weight)

#Combine the weights with the total of the Strength Score
df['Divisiveness'] = df['Strength Score'] * df['Weight']

In [76]:
#Get the total divisiveness score for each submitter across rounds and sort

scores = df.groupby('Submitter ID').agg({'Divisiveness': 'sum'}).reset_index()
scores.sort_values('Divisiveness', ascending=False, inplace=True)
scores.reset_index(inplace=True)

#Get submitter names
pd.merge(scores, competitors, left_on='Submitter ID', right_on='ID')[['Name', 'Divisiveness']]

,Name,Divisiveness
0,yousef,56.314507
1,Brian Lee,54.048118
2,sydney joseph,50.772859
3,Kelsey,43.173016
4,Bryce Curtis Miller,40.353361
5,Kevin,39.910227
6,Seth Reichert,39.034284
7,Ashton Hicks,38.524185
8,Brittany Gernhard,35.839406
9,Mackenzie Bartek,34.805445
